In [4]:
%reload_ext autoreload
%autoreload 2

# MinHash on HT books

[MinHash](https://en.wikipedia.org/wiki/MinHash) is a technique for estimating similarity of two sets quickly. Where a regular hash can fingerprint completely identical objects, MinHash is particularly useful for comparing sets that are similar but inexactly. It's used often for texts, where the *set* is comprised of the words in the text.

MinHash scales linearly, making it useful for an initial scan of similar texts before using a more computationally intensive algorithm.

This project uses `datasketch` to convert HTRC Extracted Features files to MinHashes, using a basic serialization method to write the HathiTrust volume ID and hash to file.

## Serializing Hashes

The basic function for hashing a text is `make_hash(htid, wordset)`, available in `hash_utils.py`. It takes a string name for the volume (max 30 characters) and a set of words. Currently, the set of words is unweighted by frequency. Other keyword arguments are passed on to `datasketch`, most importantant the `num_perm` parameter that allows you to increase the size of the hash for better estimation at a cost of more memory and time. You can also change the `seed` for hashing.

`make_hash` returns a byte serialization of the text, using the following format:

- seed (8bytes long long)
- length of hash (4 bytes),
- volume name (max 30 char string, padded with `\x00`)
- hashes (4 bytes each, * length)

In [6]:
from hash_utils import make_hash
h = make_hash('testid', {'it', 'was', 'the', 'best', 'of', 'times'})
h

b'\x01\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00testid\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xacPG\x182\xd8$%\ny\x02\x17k\xaf\xacI\x15\xef\x1e\x08\xdf~\x1e<\xe5\xbf\xda\nGP\xd5\x120\xbc\x1c\'\xf2\xbb^\x05Q\xee\x0e\x08\x00\xddl\x04\x1d\xe4\xb8\x14\xfc:x6u\x1e#B\x92\xb9N\x06\x15"\xb1)j\x13\x7f&\xba\x90~8q\\\xdcJ\xce\x0e\xac-Q\x8e\xbe\x02\x97\xb6?\x03\x02Q\xb0\x05\xe3+\xfd9$\xf3e#\x9fh\x10\r\x8c\xa7j\x13G\x12=#$P\xf4Q\x16a\xe3\x0c\x17\xef9\x19\x90\xc4\xe5\x0f\x03d\xdb"<\x16\x01&\xe4dm\x06\x9e\xb8\xa8,\x1a\xb001\xa4\xce\x8c\x19(\x84\x97\x0bE1H\x1bkK]\x02"-\xcea\x81\xedr\x07\x1a\xb7\x02\x0f\xd7\xb6\xd2\n\xc1\x7f\x00\r\xd2~`\x16\x17\xf4<\x172\x03g\r\xae\xd9\x88n\x9a\x1f\xb4d\x1a\xf4\xa9j\xda3.!vT\xd9\x05\xcd\xbf`\x1f4\x86\xb2\n\xcd7\'\x1c\x8f^\xcc0G\tT>\xf2_J\x1di\xd6m+\x87\xc4d%\xafD\x8bKju\xcb\'\xde=z$\xfcH/.\x98\x974\x00+|.\t\xb6\xce\xa9\x00\t6\xc0\x18o\x1b\xbd3=\x14\x8c6\xc9\x997\x10C\x89m\x1a,\x95\x0f5\xf2]\xe6\x1f\xc3\

## Crunching Hashes (Method 1: Pre-processed files)

For my input, I use a pre-crunched version of the Extracted Features dataset created for HTRC+Bookworm, documented [here](https://github.com/htrc/HTRC-Useful-Datasets/tree/master/volume-tables). Those files are tables of volumeid/token/count for 13m books and are simply used for convenience: you can request them from HTRC or simply process from the original dataset (see below).

The code for using those files is in [MakeMinHash.py](./MakeMinHash.py). It runs a single process, transforming an input H5 file into a custom data file. To parallelize, I use `GNU Parallel` on the command line. e.g.

```bash
find ../datasets/volume-tables/data/. -name '*h5' | parallel -j20 -n1 -u python MakeMinHash.py {} data/minhashes/{/.}.dat
```

## Crunching Hashes (Method 2: Extracted Features Files)



The code for converting Extracted Features files to Minhashes is in [./MakeMinHashFromIDs.py](./MakeMinHashFromIDs.py). Below is the usage doc:

In [2]:
!python MakeMinHashFromIDs.py --help

usage: MakeMinHashFromIDs.py [-h]
                             (--ids IDS [IDS ...] | --paths PATHS [PATHS ...])
                             [--num_perm NUM_PERM]
                             outdir

Process a BW file into a Minhash and save using a succinct serialization
format.

positional arguments:
  outdir                Directory to save Minhashes. File name is
                        hashes.{RANDINT}.dat.

optional arguments:
  -h, --help            show this help message and exit
  --ids IDS [IDS ...]   HathiTrust IDs to convert to MinHash. Mutually
                        exclusive with --paths.
  --paths PATHS [PATHS ...]
                        Paths to HathiTrust EF file to convert to MinHash.
                        Mutually exclusive with --ids.
  --num_perm NUM_PERM   Number of permutations for MinHash. Defaults to 128


## Deserializing Hashes

Use `deserialize_lm`. Even though the length is saved in the serialization, this basic function asks for the length of the hash. This is identical to the `num_perm` argument, `128` if the default wasn't changed.

In [13]:
from hash_utils import deserialize_lm
volid, minhash = deserialize_lm(h, 128)
volid, minhash

(b'testid', <datasketch.lean_minhash.LeanMinHash at 0x7f83d6134708>)

In my case, I save hashes to a file with the same settings, so I simply peek at the first 12 bytes of the file to get the length, then read the rest with the assumption that it is all constant-length hashes. If following the same usecase, use `HashReader`. e.g.

In [63]:
from hash_utils import HashReader
example_in = 'data/minhashes/counts.64.dat'

with HashReader(example_in) as hr:
    i = 0
    for htid, minhash in hr.hashes():
        print(htid, minhash)
        i += 1
        if i >= 5:
            break

b'uva.x030509704' <datasketch.lean_minhash.LeanMinHash object at 0x7f84485db678>
b'uva.x030509706' <datasketch.lean_minhash.LeanMinHash object at 0x7f84485db3a8>
b'uva.x030509708' <datasketch.lean_minhash.LeanMinHash object at 0x7f84485db318>
b'uva.x030509709' <datasketch.lean_minhash.LeanMinHash object at 0x7f84485db1f8>
b'uva.x030509710' <datasketch.lean_minhash.LeanMinHash object at 0x7f84485db288>


# Using the hashes

The MinHashes can be used for comparison of sets using locality-sensitive hashing, as per [`datasketch` docs](https://ekzhu.github.io/datasketch/lsh.html). `MinHashLSH` needs a threshold for 'similarity' at initialization, while `MinHashLSHForest` allows a 'top *n*' query for finding a number of similar texts. 

e.g.

In [95]:
from datasketch import MinHashLSHForest
lshf = MinHashLSHForest(num_perm=128)

errs, added = 0, 0
with HashReader(example_in) as hr:
    for htid, minhash in hr.hashes():
        try:
            lshf.add(htid, minhash)
            added += 1
        except:
            errs += 1
print(added, 'volumes added with', errs, 'errors')
lshf.index()

6551 volumes added with 28 errors
CPU times: user 932 ms, sys: 8 ms, total: 940 ms
Wall time: 940 ms


The error catching here is a quirk of my speed-optimized workflow: some files may have have been hashed twice (likely with two incomplete fingerprints). In my case, I just ignore those particular hashes and rehash from the original dataset file, but a more delicate hashing process wouldn't need to do that.

Note that MinHashLSHForest needed a call to `index` at the end. Then, you can provide a hash query to find similar texts:

In [104]:
# Use the last minhash from the loop for a query
print(htid, minhash)
print('Closest 10 volumes (unsorted) are:')
lshf.query(minhash, k=10)

b'coo.31924069731796' <datasketch.lean_minhash.LeanMinHash object at 0x7f83d60bc3a8>
Closest 10 volumes (unsorted) are:


[b'uc1.b4017365',
 b'uc1.b5173966',
 b'coo.31924069731762',
 b'coo.31924069731796',
 b'coo.31924069731770',
 b'uc1.b4450359',
 b'coo.31924069731788',
 b'uc1.b4498285',
 b'uc1.b3810741',
 b'coo.31924069731705']